In [6]:
## Word Embeddings ##
# LASER Embeddings pre-trained model

import bucc_proc as bp
import importlib
importlib.reload(bp)
from laserembeddings import Laser

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
new_df = bp.get_merge()
en_proc = bp.en_proc
zh_proc = bp.zh_proc
cosine_similarity = bp.cosine_similarity
new_df.count

Merged file exists, reading...


<bound method DataFrame.count of              ID_zh         ID_en                             Sentence_zh  \
0     zh-000000033  en-000005983               1989年以前，全球经济包含大约8亿到10亿人口。   
1     zh-000000231  en-000047360        今日全球面临的威胁是超民族的，因此也必须采取超民族的方式来应对。   
2     zh-000000272  en-000027140                   欧盟移民政策的硬伤还有一个不太显著的方面。   
3     zh-000000438  en-000065621           只有让民粹主义服务于自由主义改革，政府才能取得长久的利益。   
4     zh-000000639  en-000005169       但社会民主派必须理解为何示威的发展会独立于现有的有组织中左翼政治。   
...            ...           ...                                     ...   
1848  zh-000094590  en-000013258         事件发生后当局在尚未进行调查的情况下就匆匆掩埋了出事列车残骸。   
1849  zh-000094593  en-000061419             北方拥有丰富的自然资源，就连电力也是从北方输送到南方。   
1850  zh-000094607  en-000039373                如果利率为3%，那么年税收额必须增加15亿美元。   
1851  zh-000094611  en-000003807           五年前，叙利亚北部边陲城镇享受着土耳其高速经济增长的红利。   
1852  zh-000094633  en-000083972  在过去的一个世纪中，我们的世界发生了翻天覆地的变化——技术是其中的重要原因。   

                                            Sentence_e

In [11]:
def get_vector(sentence,lang="en",proc=True):
    if proc==True:
        if lang == "en":
            s = ' '.join(en_proc(sentence))
        elif lang == "zh":
            s = ' '.join(zh_proc(sentence))
        else:
            print('No proccessing method for this language.')
    else:
        s = sentence
    return laser.embed_sentences(s, lang=[lang])

In [12]:
laser = Laser()

In [81]:
def test(n, x=0, proc=True):
    string = ''
    
    tp=0 #true positives
    fn=0 #false negatives
    
    for i in range(x, n):
        en_vect = get_vector(new_df.iloc[i]['Sentence_en'], proc=proc)
        zh_vect = get_vector(new_df.iloc[i]['Sentence_zh'], lang="zh", proc=proc)
        this_value = cosine_similarity(en_vect, zh_vect)
        
        if i == x:
            most = this_value
            least = this_value
            most_i = x
            least_i = x
            
        elif most <= this_value:
            most = this_value
            most_i = i
            
        elif least >= this_value:
            least = this_value
            least_i = i
        
        if this_value>0.7:
            j = 'YES'
            tp+=1
        else:
            j = 'NO'
            fn+=1
        
        string = string + str(i) +  "\t{:.2f}".format(round(this_value,2)) + '\t' + j + '\n'
    
    
    string = 'Line\tSimilarity\tPair\n' + string
    string = 'Most:' + str(most_i) + "\t{:.2f}".format(most) + '\n' + string
    string = 'Least:' + str(least_i) + "\t{:.2f}".format(least) + '\n' + string
    string = 'Size:' + str(n-x) + '\tTP:' + str(tp) + '\tFN:'+ str(fn) + '\tAccuracy:' + str(round(tp/(n-x)*100,1))+'%\n' + string
    return string
            
def str_to_txt(filename, string):
    with open(filename,'w', encoding='utf-8') as file:
        file.write(string)
                
#str_to_txt('laser_output.txt',test(1853))
#str_to_txt('laser_output_np.txt',test(1853, proc=False))

In [82]:
from random import randrange


def negatest(t,n=0, proc=True):
    string = ''
    
    tn=0 #true negatives
    fp=0 #false positives
    
    start_n = n
    
    while t > n:
        x = randrange(1853)
        y = randrange(1853)
        if x != y:
            en_vect = get_vector(new_df.iloc[x]['Sentence_en'], proc=proc)
            zh_vect = get_vector(new_df.iloc[y]['Sentence_zh'], lang="zh", proc=proc)
            this_value = cosine_similarity(en_vect, zh_vect)
            
            x = str(x)
            y = str(y)
            
            if n == start_n:
                most = this_value
                least = this_value
                most_i = (x,y)
                least_i = (x,y)
            
            elif most <= this_value:
                most = this_value
                most_i = (x,y)
            
            elif least >= this_value:
                least = this_value
                least_i = (x,y)
        
            
            if this_value>0.7:
                j = 'YES'
                fp+=1
            else:
                j = 'NO'
                tn+=1
            
            string = string + '\t' + x + '\t' + y +  "\t{:.2f}".format(round(this_value,2)) + '\t' + j + '\n'
            n+=1
    
    
    string = 'Line_en\tLine_zh\tSimilarity\tPair\n' + string
    string = 'Most:' + '\t'.join(most_i) + "\t{:.2f}".format(most) + '\n' + string
    string = 'Least:' + '\t'.join(least_i) + "\t{:.2f}".format(least) + '\n' + string
    string = 'Size:' + str(t-start_n) + '\tTN:' + str(tn) + '\tFP:'+ str(fp) + '\tAccuracy:' + str(round(tn/(t-start_n)*100,1))+'%\n' + string
    return string
            
        
#str_to_txt('laser_output_negative.txt',negatest(1853))
#str_to_txt('laser_output_negative_np.txt',negatest(1853, proc=False))

In [36]:
# Setting requirement to 0.7 similarity to exclude false positives
# Line 9 is a problem, similarity close to non-pair sentences
print(new_df.iloc[9]['Sentence_en'],'\t',new_df.iloc[9]['Sentence_zh'])

Millions of people around the world will live, or die, depending on what these governments decide in December. 	 各国政府在11月的决定将影响数百万人的生死。


In [13]:
# Testing without proccessing text

def test_no_proc(n, x=0):
    k=0
    j=''
    for i in range(x, n):
        this_value = cosine_similarity( get_vector(new_df.iloc[i]['Sentence_en'], proc=False), get_vector(new_df.iloc[i]['Sentence_zh'],lang="zh",proc=False) )
        if i == x:
            most = this_value
            s_index = 0
        elif most <= this_value:
            most = this_value
            s_index = i
        
        if this_value>0.7:
            j='YES'
            k+=1
        else:
            j='NO'
        
        print('Line:',i,'\tSim:',round(this_value,2), '\tPair:', j)
        
    return s_index,most, 'Success Rate:'+str(round(k/(n-x)*100,1))+'%'
            

    
test_no_proc(50)

Line: 0 	Sim: 0.9 	Pair: YES
Line: 1 	Sim: 0.85 	Pair: YES
Line: 2 	Sim: 0.74 	Pair: YES
Line: 3 	Sim: 0.87 	Pair: YES
Line: 4 	Sim: 0.88 	Pair: YES
Line: 5 	Sim: 0.87 	Pair: YES
Line: 6 	Sim: 0.86 	Pair: YES
Line: 7 	Sim: 0.87 	Pair: YES
Line: 8 	Sim: 0.82 	Pair: YES
Line: 9 	Sim: 0.67 	Pair: NO
Line: 10 	Sim: 0.93 	Pair: YES
Line: 11 	Sim: 0.83 	Pair: YES
Line: 12 	Sim: 0.81 	Pair: YES
Line: 13 	Sim: 0.89 	Pair: YES
Line: 14 	Sim: 0.86 	Pair: YES
Line: 15 	Sim: 0.88 	Pair: YES
Line: 16 	Sim: 0.86 	Pair: YES
Line: 17 	Sim: 0.82 	Pair: YES
Line: 18 	Sim: 0.89 	Pair: YES
Line: 19 	Sim: 0.9 	Pair: YES
Line: 20 	Sim: 0.89 	Pair: YES
Line: 21 	Sim: 0.84 	Pair: YES
Line: 22 	Sim: 0.86 	Pair: YES
Line: 23 	Sim: 0.93 	Pair: YES
Line: 24 	Sim: 0.9 	Pair: YES
Line: 25 	Sim: 0.88 	Pair: YES
Line: 26 	Sim: 0.88 	Pair: YES
Line: 27 	Sim: 0.83 	Pair: YES
Line: 28 	Sim: 0.92 	Pair: YES
Line: 29 	Sim: 0.91 	Pair: YES
Line: 30 	Sim: 0.84 	Pair: YES
Line: 31 	Sim: 0.92 	Pair: YES
Line: 32 	Sim: 0.86 	P

(36, 0.9389751, 'Success Rate:98.0%')

In [46]:
from random import randrange

t = 50
n = 0

while t > n:
    x = randrange(1853)
    y = randrange(1853)
    if x != y:
        z = cosine_similarity( get_vector(new_df.iloc[x]['Sentence_en'], proc=False), get_vector(new_df.iloc[y]['Sentence_zh'],lang="zh", proc=False) )
        zz = 'YES' if z>0.7 else 'NO'
    
        print('Line EN:',x,'\tLine ZH:',y,'\tSim:',round(z,2), '\tPair:', zz)
    n+=1

Line EN: 463 	Line ZH: 724 	Sim: 0.54 	Pair: NO
Line EN: 749 	Line ZH: 1125 	Sim: 0.52 	Pair: NO
Line EN: 996 	Line ZH: 1843 	Sim: 0.51 	Pair: NO
Line EN: 503 	Line ZH: 495 	Sim: 0.5 	Pair: NO
Line EN: 1527 	Line ZH: 963 	Sim: 0.59 	Pair: NO
Line EN: 311 	Line ZH: 186 	Sim: 0.5 	Pair: NO
Line EN: 519 	Line ZH: 997 	Sim: 0.49 	Pair: NO
Line EN: 361 	Line ZH: 1120 	Sim: 0.54 	Pair: NO
Line EN: 273 	Line ZH: 1662 	Sim: 0.53 	Pair: NO
Line EN: 624 	Line ZH: 1826 	Sim: 0.54 	Pair: NO
Line EN: 916 	Line ZH: 1138 	Sim: 0.5 	Pair: NO
Line EN: 1612 	Line ZH: 1274 	Sim: 0.52 	Pair: NO
Line EN: 712 	Line ZH: 1277 	Sim: 0.5 	Pair: NO
Line EN: 529 	Line ZH: 18 	Sim: 0.52 	Pair: NO
Line EN: 643 	Line ZH: 604 	Sim: 0.52 	Pair: NO
Line EN: 98 	Line ZH: 1669 	Sim: 0.45 	Pair: NO
Line EN: 727 	Line ZH: 773 	Sim: 0.49 	Pair: NO
Line EN: 434 	Line ZH: 648 	Sim: 0.52 	Pair: NO
Line EN: 423 	Line ZH: 1614 	Sim: 0.52 	Pair: NO
Line EN: 455 	Line ZH: 1688 	Sim: 0.55 	Pair: NO
Line EN: 1385 	Line ZH: 1040 	Sim

In [ ]:
## Not processing text is better as stopwords are kept => more information 

In [ ]:
# True Positive (is a pair and classified correct)
# False Positive (is actually not a pair but classified wrong)
# True Negative (is not a pair and classified correct)
# False Negative (is actually a pair but classified wrong)

In [90]:
# Set 0.7 sim as classifying line

#np (no proc)

tn = 1853
fp = 0
tp = 1831
fn = 22
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2*(precision*recall/(precision+recall))

print('Without processing into bag of words:')
print('Precision:', round(precision*100,1), '%')
print('Recall:', round(recall*100,1), '%')
print('F1 Score:', round(f1*100,1), '%')


# with proc
tn = 1853
fp = 0
tp = 1299
fn = 554
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2*(precision*recall/(precision+recall))

print('\nWith processing into bag of words:')
print('Precision:', round(precision*100,1), '%')
print('Recall:', round(recall*100,1), '%')
print('F1 Score:', round(f1*100,1), '%')



Without processing into bag of words:
Precision: 100.0 %
Recall: 98.8 %
F1 Score: 99.4 %

With processing into bag of words:
Precision: 100.0 %
Recall: 70.1 %
F1 Score: 82.4 %
